# Define frequency bins for map-making
## A. Ordog, Feb 07, 2023

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Read in persistent RFI mask and determine fraction of channels lost

In [ ]:
RFI_mask_idx = []
RFI_mask_freq = []
with open('/home/ordoga/Python/DVA2/DATA/PersistRFImaskNew_v2.txt') as fp:    
    for line in fp:
        if line.split()[1] == 'False':
            RFI_mask_idx.append(int(line.split()[0]))
            RFI_mask_freq.append(float(line.split()[2]))

RFI_mask_idx = np.array(RFI_mask_idx)
RFI_mask_freq = np.array(RFI_mask_freq)

RFI_mask_idx.shape[0]/8250

## Write out only indices of adjusted RFI mask (done only once)

In [ ]:
#####################
save_new_mask = False
#####################
if save_new_mask:
    with open('/home/ordoga/Python/DVA2/DATA/PersistRFImaskNewJustIndexBad_v2.txt','w') as fp:    
        fp.write('index   frequency (MHz)')
        fp.writelines('\n')
        for i in range(0,len(RFI_mask_idx)):
            fp.write(str(RFI_mask_idx[i])+' '+str(RFI_mask_freq[i]))
            fp.writelines('\n')

## Function to determine how many good frequencies per bin based on starting index

In [ ]:
def make_bins(ii):

    dfbin = 0.5
    f_bin_arr = []
    nu_bin_arr = [] # number above centre of bin
    nd_bin_arr = [] # number below centre of bin

    f_bin = (freq[ii]+freq[ii+1])/2.
    while f_bin < 1030.:
        wu = np.where( (freq <= f_bin+dfbin/2.) & (freq >= f_bin) & (mask == 'True') )[0]
        wd = np.where( (freq >= f_bin-dfbin/2.) & (freq <= f_bin) & (mask == 'True') )[0]
        #w = np.where( (abs(freq - f_bin) <= dfbin/2.) & (mask == 'True') )[0]
        f_bin_arr.append(f_bin)
        nu_bin_arr.append(len(wu))
        nd_bin_arr.append(len(wd))
        f_bin = f_bin + dfbin
    print(f_bin_arr[0],f_bin_arr[-1])

    nu_bin_arr = np.array(nu_bin_arr)
    nd_bin_arr = np.array(nd_bin_arr)
    
    return f_bin_arr, nu_bin_arr, nd_bin_arr

## Plot to visualize number of good channels

In [ ]:
f_bin_arr, nu_bin_arr, nd_bin_arr = make_bins(68)

fig,ax = plt.subplots(15,1,figsize=(16,30))
for i in range(0,14):
    ax[i].scatter(f_bin_arr,nu_bin_arr,s=10,color='red',zorder=10)
    ax[i].scatter(f_bin_arr,nd_bin_arr,s=50,color='C0',zorder=1)
    #ax[i].plot(f_bin_arr,n_bin_arr)
    ax[i].set_xlim(340+i*50,340+(i+1)*50)
    ax[i].set_ylim(-0.5,3.5)
    ax[i].grid()
    ax[i].set_facecolor('lightgrey')
ax[14].scatter(f_bin_arr,nu_bin_arr,s=5,color='red',zorder=10)
ax[14].scatter(f_bin_arr,nd_bin_arr,s=20,color='C0',zorder=1)
ax[14].set_xlim(350,1030)
ax[14].set_ylim(-0.5,3.5)
ax[14].grid()
ax[14].set_facecolor('lightgrey')